Så prøver vi en gang til at få det her shizzle til at virke. Først alle import

In [2]:
import sys
import os

In [3]:
os.chdir('/home/au643300/DataHandling/')

Sætter Datahandling in i min syspath

In [4]:
sys.path.append('/home/au643300/DataHandling/')

In [5]:
from dask.distributed import Client, as_completed, LocalCluster, get_task_stream
import dask
import xarray as xr
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, fire_and_forget
import glob
import importlib
from src.data.tonetCDF import to_netcdf  

starter cluster op

In [5]:
cluster=SLURMCluster(cores=4,
                     processes=1,
                     memory="22GiB",
                     queue='q64',
                     walltime='0-00:30:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     extra=['--resources mem=20']
                    )
                    

In [8]:
client=Client(cluster)

In [9]:
cluster.adapt(minimum_jobs=1, maximum_jobs=3)

In [10]:
client

Client Scheduler: tcp://10.100.8.13:46542 Dashboard: http://10.100.8.13:8787/status,Cluster Workers: 1 Cores: 4 Memory: 23.62 GB


In [6]:
raw_path="/home/au643300/DataHandling/data/raw/"
filelist_raw=glob.glob(raw_path+'*.u')

interim_path="/home/au643300/NOBACKUP/interim/"
filelist_interim=glob.glob(interim_path+'*.nc')
for file in filelist_interim:
    size=os.path.getsize(file)
    if size < 500000000:
        os.remove(file)


In [28]:
file_only_interim=[]
file_only_raw=[]
for file in filelist_interim:
    a=os.path.basename(file)
    a=a[:-3]
    file_only_interim.append(a)
for file in filelist:
    b=os.path.basename(file)
    b=b[:-2]
    file_only_raw.append(b)

In [32]:
diff=list(set(file_only_interim)^set(file_only_raw))

In [62]:
sym_path=[]
for name in diff:
    sym_path.append("/home/au643300/DataHandling/data/raw/"+name+'.u')


In [63]:
file_path=[]
for path in a:
    file_path.append(os.readlink(path))

In [12]:
client.map(to_netcdf,file_path,resources={'mem': 5},retries=5)